In [1]:
from airtable import AirtableInterface
from dotenv import load_dotenv
from os import environ
import pandas as pd
import plotly.express as px

load_dotenv()

airt = AirtableInterface(environ["AIRTABLE_API_KEY"], environ['AIRTABLE_BASE_ID'], environ['AIRTABLE_TABLE_KEY'])

In [2]:
df = airt.get_pd_dataframe()
df['time'] = pd.to_datetime(df['time'])

def extract_website_name(website):
    if '/' in website:
        return website.split('/')[0]
    if "main.google.com" in website:
        return "gmail.com"
    return website

def clean_program_name(program_name):
    cleaned_name = program_name.strip()

    if cleaned_name.startswith('[') and cleaned_name.endswith(']'):
        cleaned_name = cleaned_name[1:-1]
    
    cleaned_name = cleaned_name.replace("'", "")

    return cleaned_name

# Apply the function to the 'program_name' column
df['program_name'] = df['program_name'].astype(str)
df['program_name'] = df['program_name'].apply(clean_program_name)

# Apply the function to the website column, and use program name where website is NaN
df['website_or_program'] = df['website'].apply(lambda x: extract_website_name(x) if pd.notna(x) else x)
df['website_or_program'].fillna(df['program_name'], inplace=True)
df['project'] = df['project'].astype(str)
df['website_or_program'] = df['website_or_program'].astype(str)

In [3]:
category_counts = df['category'].value_counts()

# Create a bar chart using Plotly for interactive visualization
fig = px.bar(category_counts, title="Time Spent on Different Categories", labels={'value':'Time Spent', 'index':'Category'})
fig.show()

In [4]:
grouped_data = df.groupby(['category', 'project']).size().reset_index(name='time_spent')

# Create a stacked bar chart using Plotly for interactive visualization
fig = px.bar(grouped_data, x='category', y='time_spent', color='project', title='Time Spent on Different Categories by Project', labels={'time_spent':'Time Spent'})
fig.show()

In [5]:
grouped_data = df.groupby(['category', 'project']).size().reset_index(name='time_spent')

# Create a stacked bar chart using Plotly for interactive visualization
fig = px.bar(grouped_data, x='project', y='time_spent', color='category', title='Time Spent on Different Projects by Category', labels={'time_spent':'Time Spent'})
fig.show()

In [6]:
grouped_data = df.groupby(['project', 'website_or_program']).size().reset_index(name='time_spent')

# Create a stacked bar chart using Plotly for interactive visualization
fig = px.bar(grouped_data, x='project', y='time_spent', color='website_or_program', 
             title='Time Spent on Different Projects by Website/Program',
             labels={'time_spent':'Time Spent', 'website_or_program':'Website/Program'})
fig.show()

In [7]:
grouped_data = df.groupby(['category', 'website_or_program']).size().reset_index(name='time_spent')

# Create a stacked bar chart using Plotly for interactive visualization
fig = px.bar(grouped_data, x='category', y='time_spent', color='website_or_program', 
             title='Time Spent on Different Categories by Website/Program',
             labels={'time_spent':'Time Spent', 'website_or_program':'Website/Program'})
fig.show()

In [8]:
# Group by 'website_or_program' and count the occurrences
website_program_counts = df['website_or_program'].value_counts()

# Create a plotly figure for a list-style chart
fig = px.bar(website_program_counts, title="Time Spent on Different Websites/Programs", labels={'value':'Time Spent', 'index':'Website/Program'})
fig.show()

In [9]:
website_program_counts = df['website_or_program'].value_counts()

# Create a plotly figure for a horizontal (list-style) bar chart
fig = px.bar(website_program_counts, orientation='h', 
             title="Time Spent on Different Websites/Programs",
             labels={'value':'Time Spent', 'index':'Website/Program'},
             height=900)  # Adjust height based on the number of items
fig.update_layout(yaxis={'categoryorder':'total ascending'})  # Sort items
fig.show()

In [11]:
df['interval'] = df['time'].dt.floor('30S')  # Grouping time into 30-second intervals

# Group by interval and website_or_program, then count the occurrences
grouped_data = df.groupby(['interval', 'website_or_program']).size().reset_index(name='frequency')

# Convert the frequency to a time duration (30 seconds for each count)
grouped_data['time_spent_seconds'] = grouped_data['frequency'] * 30

# Create a stacked bar chart
fig = px.bar(grouped_data, x='interval', y='time_spent_seconds', color='website_or_program', 
             title='Time Spent on Each Website/Program in 30-Second Intervals',
             labels={'time_spent_seconds':'Time Spent (seconds)', 'interval':'Interval', 'website_or_program':'Website/Program'})
fig.show()